The following instructions have been tried on OSX 10.11.5.
Setup on other platforms should be quite similar.

# Requirements

* python 3.5
* mongodb >= 3.2.3
* anakonda >= 4.0.8 for python 3.5 [download](https://www.continuum.io/downloads)
* other python libraries: delorean >= 0.6.0, persistent >= 4.2.1
These libraries are installed with the following command.

> pip install ...

If you do not have pip, you can install it using these [official instructions](https://pip.pypa.io/en/stable/installing/).




## Data

### Load data into mongodb

With mongodb running, import the data with the following two commands.


In [2]:
%%bash
mongoimport -d firenze_sample -c checkins --file data/firenze_checkins.json --upsert
mongoimport -d firenze_sample -c venues --file data/firenze_venues.json --upsert

2016-06-14T18:03:42.832+0300	connected to: localhost
2016-06-14T18:03:43.154+0300	imported 900 documents
2016-06-14T18:03:43.166+0300	connected to: localhost
2016-06-14T18:03:43.279+0300	imported 332 documents


### Download other data


## Train

In [4]:
%%bash
python3.5 -W ignore train.py -k_min 1 -k_step 1 -k_max 15 --runs 10 -lambdas "1" --iter 100 --rel_change 0.00001 \
    --plot --save mongo --dbname firenze_sample --query '{"city":"Firenze"}'

Process is terminated.


The above outputs files of the following form,
* [date]-logl_across_k.pdf
* [date].desc
* [date].mdl
* [date].scaler
* [date].unigrams

where [date] is the timestamp when the program finished.